In [60]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('auteurs_mouvements.csv')

In [14]:
data

,Unnamed: 0,Nom,Mouvement,summary,sum_lem,mvm_label
0,0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0
1,1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0
2,2,Carolyn Cassady,Beat Generation,"Carolyn Cassady, née Carolyn Robinson le 28 av...",Carolyn Cassady naître Carolyn Robinson 28 avr...,0
3,3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0
4,4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0
...,...,...,...,...,...,...
256,256,Maurice Beaubourg,symboliste,"Maurice Beaubourg, né le 4 décembre 1859 et mo...",Maurice Beaubourg naître 4 décembre 1859 mort ...,9
257,257,Otokar Březina,symboliste,"Otokar Březina, né Václav Ignác Jebavý le 13 s...",otokar Březina naître Václav Ignác jebavý 13 s...,9
258,258,Emanuil Popdimitrov,symboliste,"Emanuil Popdimitrov Popzahariev est un poète, ...",emanuil Popdimitrov Popzahariev poète philosop...,9
259,259,Marcel Schwob,symboliste,"Marcel Schwob, né à Chaville (Seine-et-Oise, a...",Marcel Schwob naître Chaville 23 août 1867 mor...,9


In [4]:
datab = pd.read_csv('Mouvements_wiki_cleaned.csv')

In [183]:
#juste pour ressortir toutes les prédictions "ex aequo" dans la fonction suivante

def sort_ish(L):
    mov,key = L[0]
    sorted_final=[mov]
    k=1
    n=len(L)
    while k<n:
        mov,keyy=L[k]
        if keyy<key:
            return sorted_final
        else:
            sorted_final.append(mov)
            k+=1
    return sorted_final

In [181]:
#j'ai fixé les arguments sauf name pour pvoir utiliser directement sur le dataframe avec la méthode apply (cf ci dessous)
#c'est évidemment juste pour le premier essai


def pred(name,database=data,train=datab,n=10,option='texte'):
    if option=='tf-idf':
        #avec l'option tf-idf, on se base sur les mots les plus importants pour chaque mouvement au sens 
        #de la métrique tf-idf
        
        txt = database[database['Nom']==name]['sum_lem'].values[0].split()
        #on prend le texte associé à l'auteur *name*
        
        pred={}
        #dictionnaire des prédictions
        
        m=len(train)
        
        for k in range(m):
            #pour chaque mouvement
            
            mov = train['mouvement'][k] #on récupère son nom
            L= eval(train['tf_idf'][k])[:n] #et les n premiers mots tf_idf (n paramètre de la fction)
            c=0 #compteur qui va compter combien de mots du texte associé à l'auteur (le summary wikipedia)
            #sont des mots présents dans les n tf idf
            for keyword in L:
                #pour chaque mot clé tf idf
                if keyword in txt:
                    #s'il est dans le texte associé à l'auteur, on incrémente c.
                    c+=1
            pred[mov]=c 
            #le nombre de mots est associé au mouvement dans le dictionnaire
        PRED = sorted(dico.items(), key=lambda mot_proba: mot_proba[1])      
        PRED.reverse()
        #on ordonne le dictionnaire selon les clés
        return sort_ish(PRED) #on retourne tous les mots ayant la valeur maximale ! 
    
    
    if option=='texte':
        #idem que pour tf-idf sauf qu'on s'intéresse à tout le texte lemmatisé de la page du mouvement 
        txt = database[database['Nom']==name]['sum_lem'].values[0].split()
        pred={}
        m=len(train)
        for k in range(m):
            mov = train['mouvement'][k]
            L= train['texte_lem'][k].split() #voilà la seule différence : 
            #on prend comme liste de mots L le texte_lem qui est le texte de la page wiki du mouvement.
            c=0
            for keyword in L:
                if keyword in txt:
                    c+=1
            pred[mov]=c
        PRED = sorted(pred.items(), key=lambda mot_proba: mot_proba[1])
        PRED.reverse()
        return sort_ish(PRED)


In [165]:
data['pred_txt']=data['Nom'].apply(pred)

In [182]:
data

,Unnamed: 0,Nom,Mouvement,summary,sum_lem,mvm_label,pred_txt
0,0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0,[Symbolisme_(art)]
1,1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0,[Symbolisme_(art)]
2,2,Carolyn Cassady,Beat Generation,"Carolyn Cassady, née Carolyn Robinson le 28 av...",Carolyn Cassady naître Carolyn Robinson 28 avr...,0,[Beat_Generation]
3,3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0,[Symbolisme_(art)]
4,4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0,[Beat_Generation]
...,...,...,...,...,...,...,...
256,256,Maurice Beaubourg,symboliste,"Maurice Beaubourg, né le 4 décembre 1859 et mo...",Maurice Beaubourg naître 4 décembre 1859 mort ...,9,[Symbolisme_(art)]
257,257,Otokar Březina,symboliste,"Otokar Březina, né Václav Ignác Jebavý le 13 s...",otokar Březina naître Václav Ignác jebavý 13 s...,9,[Symbolisme_(art)]
258,258,Emanuil Popdimitrov,symboliste,"Emanuil Popdimitrov Popzahariev est un poète, ...",emanuil Popdimitrov Popzahariev poète philosop...,9,[Symbolisme_(art)]
259,259,Marcel Schwob,symboliste,"Marcel Schwob, né à Chaville (Seine-et-Oise, a...",Marcel Schwob naître Chaville 23 août 1867 mor...,9,[Symbolisme_(art)]
